In [164]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import sys
%pip install census
%pip install us
from census import Census
from us import states


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [165]:
neighborhoods = gpd.read_file('Neighborhood Boundaries/geo_export_ef027b0d-4a68-4a7f-81a6-64cd0be38041.shp')
neighborhoods = neighborhoods.set_geometry("geometry")

In [166]:
# Set API key
c = Census("9e840ebf4aecf1dfee3001ca5b9df8a1c5c8e541")

In [215]:
tracts = gpd.read_file("cb_2020_44_tract_500k/cb_2020_44_tract_500k.shp")
tracts = tracts.to_crs(4269)

places = gpd.read_file('cb_2020_44_place_500k/cb_2020_44_place_500k.shp')
places = places.set_crs(4269)
providence = places.loc[places.PLACEFP == '59000']

providence.to_file("providence.shp")

In [220]:
data_dict = {'B08133_001E':'Estimate!!Aggregate travel time to work (in minutes) AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK', 
            'B08133_007E' : 'Estimate!!Aggregate travel time to work (in minutes):!!7:00 a.m. to 7:29 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_008E' : 'Estimate!!Aggregate travel time to work (in minutes):!!7:30 a.m. to 7:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_009E' : 'Estimate!!Aggregate travel time to work (in minutes):!!8:00 a.m. to 8:29 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_010E' : 'Estimate!!Aggregate travel time to work (in minutes):!!8:30 a.m. to 8:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_011E' : 'Estimate!!Aggregate travel time to work (in minutes):!!9:00 a.m. to 9:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_012E' : 'Estimate!!Aggregate travel time to work (in minutes):!!10:00 a.m. to 10:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_013E' : 'Estimate!!Aggregate travel time to work (in minutes):!!11:00 a.m. to 11:59 a.m.	AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_001E' : 'Estimate!!Total SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_007E' : 'Estimate!!Total:!!7:00 a.m. to 7:29 a.m.	SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_008E' : 'Estimate!!Total:!!7:30 a.m. to 7:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_009E' : 'Estimate!!Total:!!8:00 a.m. to 8:29 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_010E' : 'Estimate!!Total:!!8:30 a.m. to 8:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_011E' : 'Estimate!!Total:!!9:00 a.m. to 9:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_012E' : 'Estimate!!Total:!!10:00 a.m. to 10:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_013E' : 'Estimate!!Total:!!11:00 a.m. to 11:59 a.m.	SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            "avg_time_agg" : "B08133_001E/B08011_001E",
            "avg_time_to_work_7_8am" : "(B08133_007E + B08133_008E)/(B08011_007E+ B08011_008E)",
            "avg_time_to_work_8_9am": "(B08133_009E + B08133_010E)/(B08011_009E+ B08011_010E)",
            "avg_time_to_work_9_10am" : "(B08133_011E + B08133_012E)/(B08011_011E+ B08011_012E)"}

data_dict = pd.Series(data_dict, name = "Definition")
data_dict.index.name = "col_name"
data_dict = data_dict.reset_index()

,col_name,Definition
0,B08133_001E,Estimate!!Aggregate travel time to work (in mi...
1,B08133_007E,Estimate!!Aggregate travel time to work (in mi...
2,B08133_008E,Estimate!!Aggregate travel time to work (in mi...
3,B08133_009E,Estimate!!Aggregate travel time to work (in mi...
4,B08133_010E,Estimate!!Aggregate travel time to work (in mi...
5,B08133_011E,Estimate!!Aggregate travel time to work (in mi...
6,B08133_012E,Estimate!!Aggregate travel time to work (in mi...
7,B08133_013E,Estimate!!Aggregate travel time to work (in mi...
8,B08011_001E,Estimate!!Total SEX OF WORKERS BY TIME OF DEPA...
9,B08011_007E,Estimate!!Total:!!7:00 a.m. to 7:29 a.m.\tSEX ...


In [213]:
# https://api.census.gov/data/2019/acs/acs5/variables.html
# B08011_007E	Estimate!!Total:!!7:00 a.m. to 7:29 a.m.	SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK	not required	B08011_007EA, B08011_007M, B08011_007MA	0	int	B08011
# B08133_007E	Estimate!!Aggregate travel time to work (in minutes):!!7:00 a.m. to 7:29 a.m.	AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK	not required	B08133_007EA, B08133_007M, B08133_007MA	0	int	B08133

departure_times_2015 = c.acs5.state_county_tract(fields = ('NAME',"B08133_001E","B08011_001E","B08011_007E", 'B08133_007E',"B08011_008E", 
                                        'B08133_008E',"B08011_009E", 'B08133_009E',"B08011_010E", 'B08133_010E',"B08011_011E", 'B08133_011E',
                                        "B08011_012E", 'B08133_012E',"B08011_013E", 'B08133_013E'),
                                      state_fips = states.RI.fips,
                                      county_fips = "007",
                                      tract = "*",
                                      year = 2015)

departure_times_2015_df = pd.DataFrame(departure_times_2015)

departure_times_2015_df['avg_time_agg'] = departure_times_2015_df.B08133_001E/departure_times_2015_df.B08011_001E

departure_times_2015_df['avg_time_to_work_7_8am'] = (departure_times_2015_df.B08133_007E + departure_times_2015_df.B08133_008E)/(departure_times_2015_df.B08011_007E+ departure_times_2015_df.B08011_008E)

departure_times_2015_df['avg_time_to_work_8_9am'] = (departure_times_2015_df.B08133_009E + departure_times_2015_df.B08133_010E)/(departure_times_2015_df.B08011_009E+ departure_times_2015_df.B08011_010E)

departure_times_2015_df['avg_time_to_work_9_10am'] = (departure_times_2015_df.B08133_011E + departure_times_2015_df.B08133_012E)/(departure_times_2015_df.B08011_011E+ departure_times_2015_df.B08011_012E)

departure_times_2015_df['year'] = 2015

# departure_times_2015_df["GEOID"] = departure_times_2015_df['state'] + departure_times_2015_df["county"] + departure_times_2015_df['tract']

# departure_times_2015_gdf = tracts.merge(departure_times_2015_df, on = 'GEOID')

In [214]:
departure_times_2019 = c.acs5.state_county_tract(fields = ('NAME', "B08133_001E","B08011_001E","B08011_007E", 'B08133_007E',"B08011_008E", 'B08133_008E',"B08011_009E", 'B08133_009E',"B08011_010E", 'B08133_010E',"B08011_011E", 'B08133_011E',"B08011_012E", 'B08133_012E',"B08011_013E", 'B08133_013E'),
                                      state_fips = states.RI.fips,
                                      county_fips = "007",
                                      tract = "*",
                                      year = 2019)

departure_times_2019_df = pd.DataFrame(departure_times_2019)

departure_times_2019_df['avg_time_agg'] = departure_times_2019_df.B08133_001E/departure_times_2019_df.B08011_001E

departure_times_2019_df['avg_time_to_work_7_8am'] = (departure_times_2019_df.B08133_007E + departure_times_2019_df.B08133_008E)/(departure_times_2019_df.B08011_007E+ departure_times_2019_df.B08011_008E)

departure_times_2019_df['avg_time_to_work_8_9am'] = (departure_times_2019_df.B08133_009E + departure_times_2019_df.B08133_010E)/(departure_times_2019_df.B08011_009E+ departure_times_2019_df.B08011_010E)

departure_times_2019_df['avg_time_to_work_9_10am'] = (departure_times_2019_df.B08133_011E + departure_times_2019_df.B08133_012E)/(departure_times_2019_df.B08011_011E+ departure_times_2019_df.B08011_012E)

departure_times_2019_df['year'] = 2019

# departure_times_2019_df["GEOID"] = departure_times_2019_df['state'] + departure_times_2019_df["county"] + departure_times_2019_df['tract']

# departure_times_2019_gdf = tracts.merge(departure_times_2019_df, on = 'GEOID')

In [221]:
with pd.ExcelWriter('travel_times_by_departure_time_tracts.xlsx') as writer:
    data_dict.to_excel(writer, sheet_name="Data Dictionary")
    departure_times_2015_df.to_excel(writer, sheet_name="ACS_2015")
    departure_times_2019_df.to_excel(writer, sheet_name="ACS_2019")
